In [5]:
# interview_config.py

# 👉 Swap these out for any domains you like
TOPICS = ["Full Stack Development", "Data Science"]


In [6]:
# prompt_builder.py

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

def make_prompt_template():
    system_msg = f"""
You are a 🔥 dynamic interview coach 🔥 covering these topics: {TOPICS}.

Flow rules:
1. Ask **one** question at a time.
2. After user answers:
   • If correct → ask a deeper/follow-up Q on that topic.  
   • If incorrect → simplify/reframe or revisit fundamentals before moving on.
3. **After two** questions in the same topic, switch to the other topic.
4. **Occasionally** ask a crossover question requiring both topics.
5. Keep it conversational & encouraging.

Just output the next question (and any short feedback). Do **not** write any code blocks.
"""
    return ChatPromptTemplate.from_messages([
        ("system", system_msg),
        MessagesPlaceholder(variable_name="history"),
    ])


In [7]:
# memory_store.py

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

_store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in _store:
        _store[session_id] = ChatMessageHistory()
    return _store[session_id]


In [8]:
# interview_bot.py

import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq                 # or swap in your LLM
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage

# from interview_config import TOPICS
# from prompt_builder import make_prompt_template
# from memory_store import get_session_history  # simple in-memory chat history

# 1️⃣ LLM setup
load_dotenv()
llm = ChatGroq(model="Gemma2-9b-It", groq_api_key=os.getenv("GROQ_API_KEY"))

# 2️⃣ Prompt pipeline
prompt = make_prompt_template()
chain = prompt | llm

# 3️⃣ Wrap with memory
chat = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="history",
)

def ask_next(session_id: str, user_input: str = None):
    """
    Call this every turn.
    • On first call: user_input=None → bot asks Q1.
    • Afterwards: supply the candidate's answer in user_input.
    """
    history = []
    if user_input:
        history.append(HumanMessage(content=user_input))
    config = {"configurable": {"session_id": session_id}}
    return chat.invoke({"history": history}, config=config)


In [9]:
# run_interview.py

# from interview_bot import ask_next

session = "candidate_001"

# 🚀 Start (no input → first Q)
resp = ask_next(session)
print("Interviewer:", resp.content)

Interviewer: Let's get started! 

First, tell me, what are some key differences between front-end and back-end development in a full stack context?  




In [ ]:
# 🎤 Candidate answers...
resp = ask_next(session, r"All websites require front-end and back-end development. Front-end development focuses on the visual aspects of a website — the part that users see and interact with. Back-end development comprises a site's structure, system, data, and logic.")
print("Interviewer:", resp.content)



Interviewer: That's a great start!  Can you give me an example of a specific technology used in front-end development and a technology used in back-end development? 





In [11]:
candidate_answer = """
Sure! Here’s an example of each:

* **Front-End Technology:**
  **React.js** – A popular JavaScript library used to build user interfaces, especially for single-page applications. It allows developers to create reusable UI components and manage the application's state efficiently.

* **Back-End Technology:**
  **Node.js** – A JavaScript runtime built on Chrome’s V8 engine that allows developers to run JavaScript on the server side. It’s commonly used to build APIs, handle database operations, and serve dynamic content to the front end.

Let me know if you’d like to explore more technologies or examples!

"""

resp = ask_next(session, candidate_answer)
print("Interviewer:", resp.content)

Interviewer: Excellent choices! React and Node.js are both powerful tools. 

Now let's shift gears a bit to data science. What's the fundamental difference between supervised and unsupervised learning?  






In [12]:
# your_app.py
import streamlit as st
# from interview_bot import ask_next

st.set_page_config(page_title="Dynamic Interview Bot", page_icon="🤖")
st.title("🤖 Dynamic Interview Interviewer")

# --- 1. Initialize session state ---
if "session_id" not in st.session_state:
    st.session_state.session_id = "candidate_001"  # change as you like

if "messages" not in st.session_state:
    st.session_state.messages = []
    # Kick off with the first question
    first = ask_next(st.session_state.session_id, user_input=None)
    st.session_state.messages.append(
        {"role": "assistant", "content": first.content}
    )

# --- 2. Render the chat history ---
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# --- 3. Handle user input & bot reply ---
if prompt := st.chat_input("Your answer..."):
    # 3a. Display user message
    st.chat_message("user").markdown(prompt)
    st.session_state.messages.append({"role": "user", "content": prompt})

    # 3b. Get next Q or follow-up
    bot_resp = ask_next(st.session_state.session_id, user_input=prompt)
    st.chat_message("assistant").markdown(bot_resp.content)
    st.session_state.messages.append(
        {"role": "assistant", "content": bot_resp.content}
    )


2025-06-09 13:43:02.107 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-09 13:43:02.109 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-09 13:43:02.904 
  command:

    streamlit run e:\langflow_directory\langchain_venv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-06-09 13:43:02.905 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-09 13:43:02.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-09 13:43:02.909 Session state does not function when running a script without `streamlit run`
2025-06-09 13:43:02.911 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare 

In [2]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Load GROQ API Key
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

# 1. Init Model
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)

In [3]:
# 2. Memory store (RAM-like)
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [4]:
# 3. Prompt Template: Acts like the Interviewer
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an Interview Bot. Ask technical questions and give feedback based on user's past answers. Start with Python questions."),
    MessagesPlaceholder(variable_name="messages")
])

# 4. Full chain: prompt -> model
chain = prompt | model

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage, trim_messages
from operator import itemgetter

# 1️⃣ Define your trimmer (45-token “window” of the last human+assistant turns)
trimmer = trim_messages(
    max_tokens=4500,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [11]:
# 2️⃣ Wrap your existing chain so it automatically trims history first
from langchain_core.runnables import RunnablePassthrough
# rebuild the chain with trimming baked in
trimmed_chain = (
    RunnablePassthrough.assign(
        messages=itemgetter("messages") | trimmer  # take input messages through the trimmer
    )
    | prompt  # your ChatPromptTemplate
    | model   # your ChatGroq
)

In [12]:
# 3️⃣ Re-wrap with memory
interview_bot = RunnableWithMessageHistory(
    trimmed_chain,
    get_session_history,
    input_messages_key="messages"
)

In [13]:

# 4️⃣ CLI loop with trimming in action
def run_interview():
    print("🎙️ Welcome to the Python Interview Bot!\n")
    session_id = input("Enter your name (used as session id): ").strip().lower()
    config = {"configurable": {"session_id": session_id}}

    while True:
        user_input = input("\n👤 You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("\n🚪 Exiting interview. Good luck!\n")
            break

        # a) Fetch the **full** history of this session
        full_history = get_session_history(session_id).messages

        # b) Let the trimmer cut it down to the last ~45 tokens
        trimmed_history = trimmer.invoke(full_history)

        # c) Optionally inject a system instruction after 2 answers
        #    (you could even inject before trimming, but here we append after)
        messages = list(trimmed_history)  # copy trimmed list
        if sum(1 for m in full_history if isinstance(m, HumanMessage)) == 2:
            print("\n🔄 Switching to hard-level questions...")
            messages.append(SystemMessage(content="Switch to hard-level Python interview questions now."))

        # d) Now add the current user turn
        messages.append(HumanMessage(content=user_input))

        # e) Invoke the **trimmed** + **augmented** chain
        response = interview_bot.invoke({"messages": messages}, config=config)

        # f) Show the bot’s reply
        print(f"\n🤖 Bot: {response.content}")

        # g) (Optional) Print the **untrimmed** memory so you can see everything stored
        print("\n📚 Full Memory so far:")
        for m in full_history:
            role = "You" if isinstance(m, HumanMessage) else "Bot"
            print(f"{role}: {m.content}")

In [14]:
run_interview()

🎙️ Welcome to the Python Interview Bot!


🤖 Bot: Hello! I'm your Interview Bot, ready to assess your Python skills. Let's begin with a foundational question:

**What are the differences between lists and tuples in Python?**  

Please describe their characteristics and provide examples of when you might use each.  

I'll be evaluating your understanding of data structures and your ability to articulate technical concepts clearly. Good luck! 



📚 Full Memory so far:
You: 
Bot: Hello! I'm your Interview Bot, ready to assess your Python skills. Let's begin with a foundational question:

**What are the differences between lists and tuples in Python?**  

Please describe their characteristics and provide examples of when you might use each.  

I'll be evaluating your understanding of data structures and your ability to articulate technical concepts clearly. Good luck! 



🤖 Bot: That's a great start! You've accurately highlighted several key differences between lists and tuples. 

**Here ar